In [ ]:
from numpy.lib.npyio import recfromtxt
import numpy as np
import pandas as pd
import math

#Definição das classes: Treliças, Porticos e Nós

#Classe para as treliças
class Truss:
  def __init__(self, xi, yi, xf, yf, theta, L, A = 0, k = 0, m = 0):
    self.xi = xi
    self.yi = yi
    self.xf = xf
    self.yf = yf
    self.theta = theta
    self.L = L

    if A!=0:
      E = 210*10**6
      rho = 7860
      self.k = E*A/L
      self.m = rho*A*L

    elif k!=0 and m!=0:
      self.k = k
      self.m = m

#Matriz de Rigidez Local para as treliças
  def l_stifness(self):
    K = np.zeros(shape=(4, 4))
    K[0,:] = np.array([1, 0, -1, 0])
    K[2,:] = np.array([-1, 0, 1, 0])
    K *= self.k
    rotation_truss = self.rotation_truss()
    rot_stiff = np.matmul(np.matmul(np.transpose(
                rotation_truss), K), rotation_truss)
    return rot_stiff

#Matriz de Massa Local para as treliças
  def l_inertia(self):
    M = np.zeros(shape=(4, 4))
    M[0,:] = np.array([2, 0, 1, 0])
    M[2,:] = np.array([1, 0, 2, 0])
    M *= self.m/6
    rotation_truss = self.rotation_truss()
    rot_inertia = np.matmul(np.matmul(np.transpose(
                rotation_truss), M), rotation_truss)
    return rot_inertia

#Matriz de Rotação para as treliças inclinadas
  def rotation_truss(self):
    Rot = [[np.cos(self.theta), np.sin(self.theta), 0, 0],
        [- np.sin(self.theta), np.cos(self.theta), 0, 0],
        [0, 0, np.cos(self.theta), np.sin(self.theta)],
        [0, 0, -np.sin(self.theta), np.cos(self.theta)]]
    return Rot

#Classe para os porticos
class Portico:
  def __init__(self, xi, yi, xf, yf, theta, A, L, I):
    self.theta = theta
    self.E = 210*10**6
    self.A = A
    self.L = l
    self.I = I
    self.rho = 7860
    self.xi = xi
    self.yi = yi
    self.xf = xf
    self.yf = yf


#Matriz de Rigidez local para os porticos
  def l_stifness(self):
    K = np.zeros(shape=(6, 6))
    K[0,:] = self.E*self.A/self.L*np.array([1, 0, 0, -1, 0, 0])
    K[1,:] = (6*self.E*self.I/(self.L**3))*np.array([0, 2, self.L, 0, -2, self.L])
    K[2,:] = (self.E*self.I/(self.L**2))*np.array([0, 6, 4*self.L, 0, -6, 2*self.L])
    K[3,:] = (self.E*self.A/self.L)*np.array([-1, 0, 0, 1, 0, 0])
    K[4,:] = (6*self.E*self.I/(self.L**3))*np.array([0, -2, -self.L, 0, 2, -self.L])
    K[5,:] = (self.E*self.I/(self.L**2))*np.array([0, 6, 2*self.L, 0, -6, 4*self.L])
    rotation_portico = self.rotation_portico()
    rotated_K = np.matmul(np.matmul(np.transpose(
                rotation_portico), K), rotation_portico)

    return K

#Matriz de massa local para os porticos
  def l_inertia(self):
    M = np.zeros(shape=(6, 6))
    M[0,:] = self.rho*self.A/self.L*np.array([1/3, 0, 0, 1/6, 0, 0])
    M[1,:] = self.rho*self.A/self.L*np.array([0, 13/35, 11*self.L/210, 0, 9/70, -13*self.L/420])
    M[2,:] = self.rho*self.A/self.L*np.array([0, 11*self.L/210, 1*self.L**2/105, 0, 13*self.L/420, -1*self.L/140])
    M[3,:] = self.rho*self.A/self.L*np.array([0, 9/70, 13/420*self.L, 0, 13/35, -11*self.L/210])
    M[4,:] = self.rho*self.A/self.L*np.array([1/6, 0, 0, 1/3, 0, 0])
    M[5,:] = self.rho*self.A/self.L*np.array([0, -13*self.L/420, -1*self.L/140, 0, -11*self.L/210, 1*self.L**2/105])
    rotation_portico = self.rotation_portico()
    rot_inertia = np.matmul(np.matmul(np.transpose(
                rotation_portico), M), rotation_portico)
    return rot_inertia

#Matriz de rotação para os porticos inclinados
  def rotation_portico(self):
    Rot = [[np.cos(self.theta), np.sin(self.theta), 0, 0, 0, 0],
        [- np.sin(self.theta), np.cos(self.theta), 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, np.cos(self.theta), np.sin(self.theta), 0],
        [0, 0, 0, -np.sin(self.theta), np.cos(self.theta), 0],
        [0, 0, 0, 0, 0, 1]]
    return Rot

#Classe de nós, necessária para atribuir uma identidade única a cada nó (id)
class Nodes:
  def __init__(self, x, y):
    self.x = x
    self.y = y
    self.id = np.round(1/2*(x+y)*(x+y+1) + y,3)

#Definição dos Parâmetros do projeto
l = 700/1000
c = (l/2)
h = 400/1000
e = 30/1000
f = 2000
g = 180/1000
i = 250/1000
k = np.sqrt(h**2 + c**2)
m = np.sqrt(h**2 + (l-c)**2)
delta = 1/100
theta = 70
a = 550/1000
b = 150/1000
d = (h/2)

#Momentos de Inercia e Área de cada parte da estrutura de acordo com a figura do enunciado
I1 = np.pi/4*(((25.4/2)/1000)**4-((25.4/2-1.5)/1000)**4)
A1 = np.pi*(((25.4/2)/1000)**2-((25.4/2-1.5)/1000)**2)
I2 = ((15/1000)*(25/1000)**3 - ((15-2*1.2)/1000)*((25-2*1.2)/1000)**3)/12
A2 = ((15/1000)*(25/1000) - ((15-2*1.2)/1000)*((25-2*1.2)/1000))
I3 = ((20/1000)*(20/1000)**3 - ((20-2*2.5)/1000)*((20-2*2.5)/1000)**3)/12
A3 = ((20/1000)*(20/1000) - ((20-2*2.5)/1000)*((20-2*2.5)/1000))
I4 = (5/1000)*(20/1000)**3/12
A4 = (5/1000)*(20/1000)

#Agora, são definidas as treliças da estrutura
truss_1 = []
truss_2 = []
truss_3 = []
truss_4 = []
truss_5 = []
truss_6 = []
truss_7 = []
truss_8 = []

#Treliça 1
theta = 0
xi, yi = np.round(c+e-i*np.sin(math.radians(70))-g, 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
xf, yf = np.round(c+e-i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, g, A4)
truss_1.append(element)

#Treliça 2
theta = np.pi - np.arctan(i*np.cos(math.radians(70))/(-c+i*np.sin(math.radians(70))+g))
xi, yi = np.round(e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e-i*np.sin(math.radians(70))-g, 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, np.sqrt((c-i*np.sin(math.radians(70))-g)**2 + (i*np.cos(math.radians(70)))**2), A4)
truss_2.append(element)

#Treliça 3
theta = np.arctan(i*np.cos(math.radians(70))/(c-i*np.sin(math.radians(70))))
xi, yi = np.round(e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e-i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, np.sqrt((c-i*np.sin(math.radians(70)))**2 + (i*np.cos(math.radians(70)))**2), A4)
truss_3.append(element)

#Treliça 4
theta = np.pi/2 + math.radians(70)
xi, yi = np.round(c+e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e-i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, L = 250/1000, k = 200*10**3, m=0.5)
truss_4.append(element)

#Treliça 5
theta = 0
xi, yi = np.round(c+e+i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
xf, yf = np.round(c+e+i*np.sin(math.radians(70))+g, 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, g, A4)
truss_5.append(element)

#Treliça 6
theta = np.pi - np.arctan(i*np.cos(math.radians(70))/(l+e-(c+e+i*np.sin(math.radians(70)))))
xi, yi = np.round(l+e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e+i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, np.sqrt((c-i*np.sin(math.radians(70)))**2 + (i*np.cos(math.radians(70)))**2), A4)
truss_6.append(element)

#Treliça 7
theta = np.arctan(i*np.cos(math.radians(70))/(c+e+i*np.sin(math.radians(70))+g-(l+e)))
xi, yi = np.round(l+e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e+i*np.sin(math.radians(70))+g, 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, np.sqrt((c-i*np.sin(math.radians(70))-g)**2 + (i*np.cos(math.radians(70)))**2), A4)
truss_7.append(element)

#Trelica 8
theta = np.pi/2 - math.radians(70)
xi, yi = np.round(c+e, 3), np.round(h+e, 3)
xf, yf = np.round(c+e+i*np.sin(math.radians(70)), 3), np.round(h+e+i*np.cos(math.radians(70)), 3)
element = Truss(xi, yi, xf, yf, theta, L = 250/1000, k = 200*10**3, m=0.5)
truss_8.append(element)
truss_list = [truss_1, truss_2, truss_3, truss_4, truss_5, truss_6, truss_7, truss_8]

#Nessa parte é criado cada um dos pórticos presentes na estrutura
portico_1 = []
portico_2 = []
portico_3 = []
portico_4 = []
portico_5 = []
portico_6 = []
portico_7 = []
portico_8 = []
portico_9 = []
portico_10 = []
portico_11 = []
portico_12 = []
portico_13 = []

#Portico 1
theta = 0
for j in range(0, int(l/delta)):
  xi, yi = np.round(delta*j + e, 3), np.round(0, 3)
  xf, yf = np.round(delta*j + delta + e, 3), np.round(0, 3)
  if xf<l+e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_1.append(element)
  elif xf==l+e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_1.append(element)
    break
  else:
    xf = l+e
    yf = 0
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_1.append(element)

#Portico 2
theta = 0
for j in range(0, int(l/delta)):
  xi, yi = np.round(delta*j + e, 3), np.round(h, 3)
  xf, yf = np.round(delta*j + delta + e, 3), np.round(h, 3)
  if xf<l+e and yf==h:
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_2.append(element)
  elif xf==l+e and yf==h:
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_2.append(element)
    break
  else:
    xf = l+e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A2, l, I2)
    portico_2.append(element)

#Portico 3
theta = 90
for j in range(0, int(h/delta)):
  xi, yi = np.round(e, 3), np.round(delta*j, 3)
  xf, yf = np.round(e, 3), np.round(delta*j + delta, 3)
  if xf==e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_3.append(element)
  elif xf==e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_3.append(element)
    break
  else:
    xf = e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_3.append(element)

#Portico 4
theta = 90
for j in range(0, int(h/delta)):
  xi, yi = np.round(l + e, 3), np.round(delta*j, 3)
  xf, yf = np.round(l + e, 3), np.round(delta*j + delta, 3)
  if xf==l+e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_4.append(element)
  if xf==l+e and yf==h:
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_4.append(element)
    break
  else:
    xf = l+e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A2, h, I2)
    portico_4.append(element)

#Portico 5
theta = np.arctan(h/c)
for j in range(0, int(k/delta)):
  dx = delta*np.cos(theta)
  dy = np.sin(theta)
  xi, yi = np.round(dx*j + e, 3), np.round(dy*j, 3)
  xf, yf = np.round(dx*j + dx, 3), np.round(dy*j + dy, 3)
  if xf<c+e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A1, k, I1)
    portico_5.append(element)
  elif xf<c+e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A1, k, I1)
    portico_5.append(element)
    break
  else:
    xf = c+e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A1, k, I1)
    portico_5.append(element)

#Portico 6
theta = np.pi - np.arctan(h/(a-c))
for j in range(0, int(m/delta)):
  dx = delta*np.cos(theta)
  dy = delta*np.sin(theta)
  xi, yi = np.round(dx*j + a + e, 3), np.round(dy*j, 3)
  xf, yf = np.round(dx*j + dx + a + e , 3), np.round(dy*j + dy, 3)
  if c+e<xf and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A1, m, I1)
    portico_6.append(element)
  elif c+e==xf and yf==h:
    element = Portico(xi, yi, xf, yf, theta, A1, m, I1)
    portico_6.append(element)
    break
  else:
    xf = c+e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A1, m, I1)
    portico_6.append(element)

#Portico 7
theta = 90
for j in range(0, int(h/delta)):
  xi, yi = np.round(a + e, 3), np.round(delta*j, 3)
  xf, yf = np.round(a + e, 3), np.round(delta*j + delta, 3)
  if xf<a+e and yf<h:
    element = Portico(xi, yi, xf, yf, theta, A1, h, I1)
    portico_7.append(element)
  if xf==a+e and yf==h:
    element = Portico(xi, yi, xf, yf, theta, A1, h, I1)
    portico_7.append(element)
    break
  else:
    xf = a+e
    yf = h
    element = Portico(xi, yi, xf, yf, theta, A1, h, I1)
    portico_7.append(element)

#Portico 8
theta = 0
for j in range(0, int(e/delta)):
  xi, yi = np.round(delta*j, 3), np.round(0, 3)
  xf, yf = np.round(delta*j + delta, 3), np.round(0, 3)
  if xf<e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_8.append(element)
  elif xf==e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_8.append(element)
    break
  else:
    xf = e
    yf = 0
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_8.append(element)

#Portico 9
theta = 0
for j in range(0, int(e/delta)):
  xi, yi = np.round(delta*j, 3), np.round(d, 3)
  xf, yf = np.round(delta*j + delta, 3), np.round(d, 3)
  if xf<e and yf==d:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_9.append(element)
  elif xf==e and yf==d:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_9.append(element)
    break
  else:
    xf = e
    yf = d
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_9.append(element)

#Portico 10
theta = 90
for j in range(0, int(e/delta)):
  xi, yi = np.round(e, 3), np.round(h + delta*j, 3)
  xf, yf = np.round(e, 3), np.round(h + delta*j + delta, 3)
  if xf==e and yf<h+e:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_10.append(element)
  elif xf==e and yf==h+e:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_10.append(element)
    break
  else:
    xf = e
    yf = h+e
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_10.append(element)

#Portico 11
theta = 0
for j in range(0, int(e/delta)):
  xi, yi = np.round(e + l + delta*j, 3), np.round(0, 3)
  xf, yf = np.round(e + l + delta*j + delta, 3), np.round(0, 3)
  if xf<l+2*e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_11.append(element)
  elif xf==l+2*e and yf==0:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_11.append(element)
    break
  else:
    xf = l+2*e
    yf = 0
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_11.append(element)

#Portico 12
theta = 0
for j in range(0, int(e/delta)):
  xi, yi = np.round(e + l + delta*j, 3), np.round(d, 3)
  xf, yf = np.round(e + l + delta*j + delta, 3), np.round(d, 3)
  if xf<l+2*e and yf==d:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_12.append(element)
  elif xf==l+2*e and yf==d:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_12.append(element)
    break
  else:
    xf = l+2*e
    yf = d
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_12.append(element)

#Portico 13
theta = 90
for j in range(0, int(e/delta)):
  xi, yi = np.round(e+l, 3), np.round(h + delta*j, 3)
  xf, yf = np.round(e+l, 3), np.round(h + delta*j + delta, 3)
  if xf==l+e and yf<h+e:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_13.append(element)
  elif xf==l+e and yf==h+e:
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_13.append(element)
    break
  else:
    xf = l+e
    yf = h+e
    element = Portico(xi, yi, xf, yf, theta, A3, e, I3)
    portico_13.append(element)
portico_list = [portico_1, portico_2, portico_3, portico_4, portico_5, portico_6, portico_7, portico_8, portico_9, portico_10, portico_11, portico_12, portico_13]

#É utilizado pandas dataframe para conseguir identificar cada indice de maneira mais eficiente
global_M = pd.DataFrame()
global_K = pd.DataFrame()


#Construindo as matrizes M e K para os porticos
for portico in portico_list:
  for element in portico:
    node1 = Nodes(element.xi, element.yi)
    node2 = Nodes(element.xf, element.yf)
    M = element.l_inertia()
    K = element.l_stifness()
    index = [f'u_{node1.id}', f'v_{node1.id}', f'theta_{node1.id}', f'u_{node2.id}', f'v_{node2.id}', f'theta_{node2.id}']
    local_M = pd.DataFrame(M, index = index, columns = index)
    global_M = pd.concat([global_M, local_M]).groupby(level=0).sum()
    local_K = pd.DataFrame(K, index = index, columns = index)
    global_K = pd.concat([global_K, local_K]).groupby(level=0).sum()

#Construindo as matrizes M e K para as treliças
for n in range(len(truss_list)):
  for element in truss_list[n]:
    node1 = Nodes(element.xi, element.yi)
    node2 = Nodes(element.xf, element.yf)
    M = element.l_inertia()
    K = element.l_stifness()
    index = [f'u_{node1.id}', f'v_{node1.id}', f'u_{node2.id}', f'v_{node2.id}']
    local_M = pd.DataFrame(M, index = index, columns = index)
    global_M = pd.concat([global_M, local_M]).groupby(level=0).sum()
    local_K = pd.DataFrame(K, index = index, columns = index)
    global_K = pd.concat([global_K, local_K]).groupby(level=0).sum()

#Função para calcular as frequências naturais e os modos de vibração
def frequencies_and_modes(inertia_matrix, stiffness_matrix):
    # Calcula as frequências naturais e modos de vibração
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.inv(inertia_matrix) @ stiffness_matrix)
    sorted_indices = np.argsort(eigenvalues)
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    selected_eigenvalues = np.real(sorted_eigenvalues[:6])
    selected_eigenvectors = np.real(sorted_eigenvectors[:, :6])
    return selected_eigenvalues, selected_eigenvectors

frequencies, modes = frequencies_and_modes(global_M.to_numpy(), global_K.to_numpy())
print("Frequências Naturais:")
for i, freq in enumerate(frequencies):
    print("Frequência {}: {:.4f} Hz".format(i+1, freq))
print("\nModos de Vibração:")
for i, mode in enumerate(modes.T):
    print("Modo {}: {}".format(i+1, mode))